# 环境准备

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
import random

from dataloader import get_data
from utils import *
from config import params
from models.texbat_model import Generator, Discriminator, DHead, QHead

In [10]:
# Set random seed for reproducibility.
seed = 1123
random.seed(seed)
torch.manual_seed(seed)
print("Random Seed: ", seed)

# Use GPU if available.
device = torch.device("cuda:0" if(torch.cuda.is_available()) else "cpu")
print(device, " will be used.\n")

Random Seed:  1123
cuda:0  will be used.



# 加载数据及训练好的模型

In [11]:
log = torch.load("/home/yhang/GAN/InfoGAN-PyTorch/checkpoint/demo8/model_epoch_50_TEXBAT_learningrate_0")
print(log["params"])

{'batch_size': 128, 'num_epochs': 100, 'D_learning_rate': 1.3e-06, 'G_learning_rate': 0.0002, 'beta1': 0.5, 'beta2': 0.999, 'save_epoch': 25, 'dataset': 'TEXBAT', 'num_z': 256, 'num_dis_c': 0, 'dis_c_dim': 0, 'num_con_c': 8}


In [12]:
discriminator = Discriminator().to(device)
discriminator.load_state_dict(log["discriminator"])
netD = DHead().to(device)
netD.load_state_dict(log["netD"])
discriminator.eval()
netD.eval()


DHead(
  (dis): Linear(in_features=128, out_features=1, bias=True)
)

In [17]:
dataloader = get_data("TEXBAT", params["batch_size"], "test")
correct = 0
total = 0
TP = 0 # 真正例
FP = 0 # 假正例
TN = 0 # 真反例
FN = 0 # 假反例
threshold = 0.5

with torch.no_grad():  # 在评估模式下，我们不需要计算梯度
    for data, labels in dataloader:
        data, labels = data.to(device), labels.to(device)
        outputs = discriminator(data)
        outputs = netD(outputs)
        predicted = (outputs > threshold).float()  # 使用0.5作为阈值来获取预测的标签
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        TP += (labels > threshold).sum().item()
        FP += (predicted > labels).sum().item()
        TN += (labels <= threshold).sum().item()
        FN += (predicted < labels).sum().item()

# 准确率
accuracy = 100 * correct / total
# 真阳性率/召回率(找的全)
TPR = TP / (TP + FN)
# 真阴性率
TNR = TN / (TN + FP)
# 精确率
Precision = TP / (TP + FP)
# 假阳率(误认为是真信号)
FPR = FP / (FP + TN)
# 假阴率(误认为是欺骗信号)
FNR = FN / (FN + TP)
print('Accuracy of the network on the test data: %f %%' % accuracy)
print('TPR/Recall of the network on the test data: %f %%' % TPR)
print('TNR of the network on the test data: %f %%' % TNR)
print('Precision of the network on the test data: %f %%' % Precision)
print('FPR of the network on the test data: %f %%' % FPR)
print('FNR of the network on the test data: %f %%' % FNR)


Accuracy of the network on the test data: 98.695691 %
TPR/Recall of the network on the test data: 0.979451 %
TNR of the network on the test data: 0.994877 %
Precision of the network on the test data: 0.994849 %
FPR of the network on the test data: 0.005123 %
FNR of the network on the test data: 0.020549 %


In [20]:
TP = 55356 # 真正例
FP = 6084 # 假正例
TN = 86690 # 真反例
FN = 12310 # 假反例


accuracy = 100 * (TP + TN) / (TP + TN + FP + FN)
# 真阳性率/召回率(找的全)
TPR = 100 * TP / (TP + FN)
# 真阴性率
TNR = 100 * TN / (TN + FP)
# 假阳率(误认为是真信号)
FPR = 100 * FP / (FP + TN)
# 假阴率(误认为是欺骗信号)
FNR = 100 * FN / (FN + TP)

print('Accuracy of the network on the test data: %f %%' % accuracy)
print('TPR/Recall of the network on the test data: %f %%' % TPR)
print('TNR of the network on the test data: %f %%' % TNR)
print('FPR of the network on the test data: %f %%' % FPR)
print('FNR of the network on the test data: %f %%' % FNR)

Accuracy of the network on the test data: 88.535278 %
TPR/Recall of the network on the test data: 81.807703 %
TNR of the network on the test data: 93.442128 %
FPR of the network on the test data: 6.557872 %
FNR of the network on the test data: 18.192297 %


In [1]:
import torch
torch.cuda.is_available()
# 返回True 接着用下列代码进一步测试
torch.zeros(1).cuda()


tensor([0.], device='cuda:0')

In [8]:
import numpy as np
data = np.array([1.0, 2.0])
print(data)
np.savetxt('./plotData/a.txt', data)

[1. 2.]
